<table style="border: none; border-collapse: collapse;">
    <tr>
        <td style="width: 20%; vertical-align: middle; padding-right: 10px;">
            <img src="https://i.imgur.com/nt7hloA.png" width="100">
        </td>
        <td style="width: 2px; text-align: center;">
            <font color="#0030A1" size="7">|</font><br>
            <font color="#0030A1" size="7">|</font>
        </td>
        <td>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="5">
                <b>Dirección de Docencia y Estudiantes Quito</b>
            </font> </p>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="4">
                Optimización de la Gestión Académica &bull; Generación Eficiente de Reportes
            </font></p>
            <p style="font-style: oblique;"><font color="#0030A1" size="3">
                Nicolás Nolivos &bull; 04/2025
            </font></p>
        </td>  
    </tr>
</table>

---
# <font color='264CC7'> Automatización de Reporte de Inscritos, Aceptados y Matriculados   </font>

> Este notebook automatiza la consolidación, transformación y visualización de datos relacionados con los procesos de inscripción, admisión y matrícula, permitiendo un reporte unificado, estético y preciso por facultad y programa académico.

---

## <font color='264CC7'>  Paquetes necesarios  

In [ ]:
from google.colab import files
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.styles import Alignment, Font, PatternFill, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.formatting.rule import ColorScaleRule
from openpyxl import load_workbook

## </font> <font color='264CC7'>  Subir Archivos  </font>

###  </font> <font color='264CC7'> Subir el Archivo base "Inscritos, Admitidos y Matriculados..."  </font>

In [ ]:
# Subir archivos
print("🔼 Sube el archivo de inscritos...")
inscritos = files.upload()
nombre_inscritos = list(inscritos.keys())[0]

🔼 Sube el archivo de inscritos...


Saving Inscritos, admitidos y matriculados al 30-04-2025.xlsx to Inscritos, admitidos y matriculados al 30-04-2025 (1).xlsx


###  </font> <font color='264CC7'> Subir un Archivo Excel con las siguientes hojas y columnas:  </font>



| 🗂️ Hoja              | 📋 Columnas requeridas                            |
|----------------------|--------------------------------------------------|
| **Grado**            | FACULTAD, CARRERA, META, META PRESUPUESTO       |
| **PUCETEC**          | TECNOLOGÍA, META, META PRESUPUESTO              |
| **POSGRADO HÍBRIDO** | CARRERA, META, META PRESUPUESTO                 |
| **POSGRADO VIRTUAL** | CARRERA, META, META PRESUPUESTO                 |

In [ ]:
print("🔼 Ahora sube el archivo de metas...")
metas = files.upload()
nombre_metas = list(metas.keys())[0]

🔼 Ahora sube el archivo de metas...


Saving metas.xlsx to metas (1).xlsx


###  </font> <font color='264CC7'>Validar que las columnas requeridas existan en las bases de datos  </font>

In [ ]:
xls_inscritos = pd.ExcelFile(nombre_inscritos)
xls_metas = pd.ExcelFile(nombre_metas)
fecha_ingresar= str(input("Ingresa el periodo lectivo de la base Inscritos (Ejemplo: 2025-01): "))

In [ ]:
# Columnas requeridas por el código en la base de inscritos
columnas_requeridas_inscritos = [
    'Facultad_Desc',       # se usa para generar 'DOMINIO'
    'Program_Desc',        # se renombra a 'Carrera_Columna'
    'Matriculado',        # usado directamente
    'Aceptado_por_institucion',
    'Recaudación'
]

# Columnas requeridas por el código en la base de metas
columnas_requeridas_metas_por_hoja = {
    'GRADO': ['FACULTAD', 'CARRERA', 'META', 'META PRESUPUESTO'],
    'PUCETEC': ['TECNOLOGÍA', 'META', 'META PRESUPUESTO'],
    'POSGRADO HÍBRIDO': ['CARRERA', 'META', 'META PRESUPUESTO'],
    'POSGRADO VIRTUAL': ['CARRERA', 'META', 'META PRESUPUESTO']
}

errores = []

for hoja_inscrito in xls_inscritos.sheet_names:
    nombre_meta = hoja_inscrito.replace(fecha_ingresar, "").strip()
    nombre_meta_upper = nombre_meta.upper()

    # Verificar columnas en hoja de inscritos
    df_base = pd.read_excel(xls_inscritos, sheet_name=hoja_inscrito)
    columnas_inscritos_actuales = df_base.columns.str.strip().tolist()

    for col in columnas_requeridas_inscritos:
        if col not in columnas_inscritos_actuales:
            errores.append(f"Falta columna '{col}' en hoja de inscritos: '{hoja_inscrito}'")

    # Verificar columnas en hoja de metas correspondiente
    if nombre_meta_upper in columnas_requeridas_metas_por_hoja:
        df_metas = pd.read_excel(xls_metas, sheet_name=nombre_meta)
        columnas_metas_actuales = df_metas.columns.str.strip().tolist()

        for col in columnas_requeridas_metas_por_hoja[nombre_meta_upper]:
            if col not in columnas_metas_actuales:
                errores.append(f"Falta columna '{col}' en hoja de metas: '{nombre_meta}'")
    else:
        errores.append(f"No se reconoce la hoja de metas '{nombre_meta}' (esperadas: {list(columnas_requeridas_metas_por_hoja.keys())})")
print()
# Mostrar resultados de validación
if errores:
    print("Se encontraron los siguientes errores antes de ejecutar el procesamiento:\n")
    for error in errores:
        print(" -", error)
else:
    print("Todas las columnas requeridas están presentes. Puedes continuar con el procesamiento.")


Ingresa el periodo lectivo de la base Inscritos (Ejemplo: 2025-01): 2025-02

Todas las columnas requeridas están presentes. Puedes continuar con el procesamiento.


## </font> <font color='264CC7'> Transformación Archivos  </font>

###   </font> <font color='264CC7'> Creación Diccionario Facultad y Entrada  </font>

In [ ]:
dominio_a_facultad = {
    "Facultad Cs de la Educación": "CIENCIAS DE LA EDUCACIÓN",
    "Facultad Ingeniería": "FACULTAD DE INGENIERÍA",
    "Hábitat, infraestructura y mov": "HABITAT",
    "Identid_educ_cult_com y valor": "IDENTIDADES",
    "Manejo Sosten de Rec Naturales":"MANEJO RECURSOS NATURALES",
    "PUCE Virtual":"PUCE VIRTUAL",
    "Pol eco_comp inst_inn_emp_prod":"POLÍTICA ECONÓMICA",
    "Pol y Der p/part soc_est rel j":"POLÍTICA Y DERECHO",
    "Vida digna_salud int - Cs Salu":"VIDA Y SALUD"
}
facultad_carrera_entrada = {
    ("HABITAT", "Arquitectura"): "HABITAT 1",
    ("HABITAT", "Artes Visuales"): "HABITAT 1",
    ("HABITAT", "Diseño Gráfico"): "HABITAT 1",
    ("HABITAT", "Ingeniería Civil"): "HABITAT 2",
    ("HABITAT", "Sistemas de Información"): "HABITAT 2",

    ("IDENTIDADES", "Comunicación"): "IDENTIDADES 1",
    ("IDENTIDADES", "Educación Básica - Semipres."): "IDENTIDADES 1",
    ("IDENTIDADES", "Pedagogía Musical - Semipresencial"): "IDENTIDADES 1",
    ("IDENTIDADES", "Filosofía"): "IDENTIDADES 2",
    ("IDENTIDADES", "Teología"): "IDENTIDADES 2",

    ("MANEJO RECURSOS NATURALES", "Biología"): "MANEJO RECURSOS NATURALES 1",
    ("MANEJO RECURSOS NATURALES", "Ciencias Biomédicas"): "MANEJO RECURSOS NATURALES 1",

    ("POLÍTICA ECONÓMICA", "Administración de Empresas"): "POLÍTICA ECONÓMICA",
    ("POLÍTICA ECONÓMICA", "Contabilidad y Auditoría"): "POLÍTICA ECONÓMICA",
    ("POLÍTICA ECONÓMICA", "Economía"): "POLÍTICA ECONÓMICA",
    ("POLÍTICA ECONÓMICA", "Finanzas"): "POLÍTICA ECONÓMICA",
    ("POLÍTICA ECONÓMICA", "Marketing"): "POLÍTICA ECONÓMICA",
    ("POLÍTICA ECONÓMICA", "Negocios Internacionales"): "POLÍTICA ECONÓMICA",

    ("POLÍTICA Y DERECHO", "Derecho SP"): "POLÍTICA Y DERECHO",
    ("POLÍTICA Y DERECHO", "Relaciones Internacionales"): "POLÍTICA Y DERECHO",
    ("POLÍTICA Y DERECHO", "Derecho"): "POLÍTICA Y DERECHO",
    ("POLÍTICA Y DERECHO", "Computación"): "POLÍTICA Y DERECHO",
    ("POLÍTICA Y DERECHO", "Contabilidad y Auditoría"): "POLÍTICA Y DERECHO",
    ("POLÍTICA Y DERECHO", "Administración de Empresas"): "POLÍTICA Y DERECHO",

    ("PUCE VIRTUAL", "Educación Básica"): "PUCE VIRTUAL",
    ("PUCE VIRTUAL", "Educación Inicial"): "PUCE VIRTUAL",
    ("PUCE VIRTUAL", "Ing Ciberseg y Gest Tec Inform"): "PUCE VIRTUAL",
    ("PUCE VIRTUAL", "Ing Redes próx Gener e Inf Nub"): "PUCE VIRTUAL",

    ("VIDA Y SALUD", "Enfermería"): "VIDA Y SALUD 1",
    ("VIDA Y SALUD", "Fisioterapia"): "VIDA Y SALUD 1",
    ("VIDA Y SALUD", "Laboratorio Clínico"): "VIDA Y SALUD 1",
    ("VIDA Y SALUD", "Medicina Veterinaria"): "VIDA Y SALUD 1",
    ("VIDA Y SALUD", "Nutrición y Dietética"): "VIDA Y SALUD 1",
    ("VIDA Y SALUD", "Psicología Clínica"): "VIDA Y SALUD 1",
    ("VIDA Y SALUD", "Psicología Organizacional"): "VIDA Y SALUD 1",
    ("VIDA Y SALUD", "Medicina"): "VIDA Y SALUD 2",
    ("VIDA Y SALUD", "Odontología"): "VIDA Y SALUD 2"
}


### </font> <font color='264CC7'> Procesamiento de Tablas  </font>


In [ ]:
# Diccionario de hojas de salida
hojas_final = {}
# Procesar cada hoja
for hoja_inscrito in xls_inscritos.sheet_names:
    print(f"Procesando hoja de inscritos: {hoja_inscrito}")

    # Ajuste para nombre de hoja de metas
    nombre_meta = hoja_inscrito.replace(fecha_ingresar, "").strip()

    # Leer hojas
    df_base = pd.read_excel(xls_inscritos, sheet_name=hoja_inscrito)
    df_metas = pd.read_excel(xls_metas, sheet_name=nombre_meta)

    # Limpiar columnas
    df_base.columns = df_base.columns.str.strip()
    df_metas.columns = df_metas.columns.str.strip()

    # Renombrar columnas base
    df_base.rename(columns={
        'Facultad_Desc': 'DOMINIO',
        'Program_Desc': 'Carrera_Columna',
        'Matriculado ': 'Matriculado'
    }, inplace=True)

    # Crear columna FACULTAD a partir del DOMINIO original
    df_base['FACULTAD'] = df_base['DOMINIO'].map(dominio_a_facultad).fillna(df_base['DOMINIO'])



    if nombre_meta.upper() == 'GRADO':
        # Crear columna FACULTAD con el diccionario (sin tocar DOMINIO original)
        df_base['FACULTAD'] = df_base['DOMINIO'].map(dominio_a_facultad).fillna(df_base['DOMINIO'])

        # Agrupar por FACULTAD y carrera
        agrupado = df_base.groupby(['FACULTAD', 'Carrera_Columna']).agg(
            INSCRITOS=('Carrera_Columna', 'count'),
            ADMITIDOS=('Aceptado_por_institucion', lambda x: (x == 35).sum()),
            MATRICULADOS=('Matriculado', lambda x: (x == 1).sum()),
            RECAUDACIÓN=('Recaudación', 'sum'),
            DOMINIO=('DOMINIO', 'first')  # mantener para cruce con entrada
        ).reset_index()

        # Renombrar Carrera_Columna a CARRERA
        agrupado.rename(columns={'Carrera_Columna': 'CARRERA'}, inplace=True)

        # Hacer merge con las metas
        df_full = pd.merge(agrupado, df_metas, how='left', on=['FACULTAD', 'CARRERA'])

        # Volver a calcular columna ENTRADA después del merge
        df_full['ENTRADA'] = df_full.apply(
            lambda row: facultad_carrera_entrada.get((row['FACULTAD'], row['CARRERA']), ''),
            axis=1
        )

        # Calcular META FACULTAD
        meta_facultad = df_full.groupby('FACULTAD')['META'].sum().reset_index()
        df_full = pd.merge(df_full, meta_facultad, on='FACULTAD', suffixes=('', '_FACULTAD'))
        df_full.rename(columns={'META_FACULTAD': 'META FACULTAD'}, inplace=True)

        # Calcular métricas
        df_full['% CUMPLIMIENTO POR FACULTAD'] = np.where(
            df_full['META FACULTAD'] > 0,
            (df_full['INSCRITOS'] / df_full['META FACULTAD'] * 100).round(0).astype(str) + '%',
            '0%'
        )

        df_full['PROMEDIO PAGO'] = np.where(
            df_full['MATRICULADOS'] > 0,
            df_full['RECAUDACIÓN'] / df_full['MATRICULADOS'],
            0
        )

        df_full['% CUMPLIMIENTO POR CARRERA'] = np.where(
            df_full['META'] > 0,
            (df_full['MATRICULADOS'] / df_full['META'] * 100).round(0).astype(str) + '%',
            '0%'
        )

        df_full['% CUMPLIMIENTO PRESUPUESTO'] = np.where(
            df_full['META PRESUPUESTO'] > 0,
            (df_full['RECAUDACIÓN'] / df_full['META PRESUPUESTO'] * 100).round(0).astype(str) + '%',
            '0%'
        )

        # Orden final de columnas
        columnas_finales = [
            'FACULTAD', 'META FACULTAD', 'ENTRADA', 'CARRERA', 'META',
            'INSCRITOS', 'ADMITIDOS', 'MATRICULADOS', 'RECAUDACIÓN', 'PROMEDIO PAGO',
            '% CUMPLIMIENTO POR CARRERA', '% CUMPLIMIENTO POR FACULTAD',
            'META PRESUPUESTO', '% CUMPLIMIENTO PRESUPUESTO'
        ]

        df_final = df_full[columnas_finales]
        hojas_final[hoja_inscrito] = df_final

    else:
        if 'FACULTAD' in df_base.columns:
          df_base.drop(columns=['FACULTAD'], inplace=True)

        df_base.rename(columns={
            'DOMINIO': 'FACULTAD'
        }, inplace=True)

        # Definir la columna clave de carrera según el tipo de hoja
        carrera_key = 'TECNOLOGÍA' if nombre_meta.upper() == 'PUCETEC' else 'CARRERA'

        # Agrupar usando DOMINIO original (sin aplicar el diccionario)
        agrupado = df_base.groupby(['FACULTAD', 'Carrera_Columna']).agg(
            INSCRITOS=('Carrera_Columna', 'count'),
            ADMITIDOS=('Aceptado_por_institucion', lambda x: (x == 35).sum()),
            MATRICULADOS=('Matriculado', lambda x: (x == 1).sum()),
            RECAUDACIÓN=('Recaudación', 'sum')
        ).reset_index()

        # Renombrar Carrera_Columna a su nombre correspondiente (CARRERA o TECNOLOGÍA)
        agrupado.rename(columns={'Carrera_Columna': carrera_key}, inplace=True)

        # Validar columna en metas
        if carrera_key not in df_metas.columns:
            raise KeyError(f"La hoja de metas '{nombre_meta}' no tiene la columna '{carrera_key}'")

        df_metas = df_metas.drop_duplicates(subset=[carrera_key])
        df_full = pd.merge(agrupado, df_metas, how='left', on=carrera_key)

        # Calcular métricas
        df_full['PROMEDIO PAGO'] = np.where(df_full['MATRICULADOS'] > 0,
                                            df_full['RECAUDACIÓN'] / df_full['MATRICULADOS'], 0)
        df_full['% CUMPLIMIENTO POR CARRERA'] = np.where(df_full['META'] > 0,
                                                        (df_full['MATRICULADOS'] / df_full['META'] * 100).round(0).astype(str) + '%',
                                                        '0%')
        df_full['% CUMPLIMIENTO PRESUPUESTO'] = np.where(df_full['META PRESUPUESTO'] > 0,
                                                        (df_full['RECAUDACIÓN'] / df_full['META PRESUPUESTO'] * 100).round(0).astype(str) + '%',
                                                        '0%')

        columnas_finales = [
            'FACULTAD', carrera_key, 'META', 'INSCRITOS', 'ADMITIDOS',
            'MATRICULADOS', 'RECAUDACIÓN', 'PROMEDIO PAGO',
            '% CUMPLIMIENTO POR CARRERA', 'META PRESUPUESTO', '% CUMPLIMIENTO PRESUPUESTO'
        ]

        if nombre_meta.upper() == 'PUCETEC':
            columnas_finales.remove('FACULTAD')

        df_final = df_full[columnas_finales]
        hojas_final[hoja_inscrito] = df_final


# Guardar Excel final
archivo_nombre = "REPORTE_MATRICULADOS_SIN_FORMATO.xlsx"
with pd.ExcelWriter(archivo_nombre, engine='openpyxl') as writer:
    for hoja, df in hojas_final.items():
        df.to_excel(writer, sheet_name=hoja, index=False)

#files.download(archivo_nombre)

Procesando hoja de inscritos: GRADO 2025-02
Procesando hoja de inscritos: PUCETEC 2025-02
Procesando hoja de inscritos: POSGRADO HÍBRIDO 2025-02
Procesando hoja de inscritos: POSGRADO VIRTUAL 2025-02


###  </font> <font color='264CC7'> Formato del Reporte  </font>

####  </font> <font color='264CC7'> Ajustes de celdas  </font>

In [ ]:
# Cargar archivo fuente
ruta_archivo = "REPORTE_MATRICULADOS_SIN_FORMATO.xlsx"
wb = openpyxl.load_workbook(ruta_archivo)

# Función para combinar y centrar valores repetidos
def combinar_y_centrar(sheet, col_name, header_row=2):
    headers = [cell.value for cell in sheet[header_row]]
    if col_name not in headers:
        return
    col_idx = headers.index(col_name) + 1
    prev_value, start_row = None, None
    for row in range(header_row + 1, sheet.max_row + 1):
        value = sheet.cell(row=row, column=col_idx).value
        if value != prev_value:
            if start_row and row - start_row > 1:
                sheet.merge_cells(start_row=start_row, start_column=col_idx, end_row=row-1, end_column=col_idx)
                sheet.cell(start_row, col_idx).alignment = Alignment(horizontal='center', vertical='center')
            start_row, prev_value = row, value
    if start_row and sheet.max_row - start_row >= 1:
        sheet.merge_cells(start_row=start_row, start_column=col_idx, end_row=sheet.max_row, end_column=col_idx)
        sheet.cell(start_row, col_idx).alignment = Alignment(horizontal='center', vertical='center')

# Función para agregar fila TOTAL con valores únicos de META FACULTAD
def agregar_fila_total_unicos(df, hoja_nombre):
    headers = df.columns.tolist()
    total_row = {}

    for col in headers:
        if col == 'PROMEDIO PAGO':
            total_row[col] = df['RECAUDACIÓN'].sum() / df['MATRICULADOS'].sum() if df['MATRICULADOS'].sum() > 0 else 0
        elif isinstance(df[col].iloc[0], str) and '%' in df[col].iloc[0]:
            total_row[col] = ''
        elif pd.api.types.is_numeric_dtype(df[col]):
            if col == 'META FACULTAD':
                total_row[col] = df[['FACULTAD', col]].drop_duplicates()[col].sum()
            else:
                total_row[col] = df[col].sum()
        else:
            total_row[col] = ''

    total_row[headers[0]] = 'TOTAL'

    # Porcentajes
    if 'META' in df.columns and 'MATRICULADOS' in df.columns and df['META'].sum() > 0:
        total_row['% CUMPLIMIENTO POR CARRERA'] = f"{(df['MATRICULADOS'].sum() / df['META'].sum() * 100):.0f}%"
    if hoja_nombre.upper().startswith("GRADO") and 'META FACULTAD' in df.columns:
        suma_mf = df[['FACULTAD', 'META FACULTAD']].drop_duplicates()['META FACULTAD'].sum()
        if suma_mf > 0:
            total_row['% CUMPLIMIENTO POR FACULTAD'] = f"{(df['INSCRITOS'].sum() / suma_mf * 100):.0f}%"
    if 'META PRESUPUESTO' in df.columns and df['META PRESUPUESTO'].sum() > 0:
        total_row['% CUMPLIMIENTO PRESUPUESTO'] = f"{(df['RECAUDACIÓN'].sum() / df['META PRESUPUESTO'].sum() * 100):.0f}%"

    return pd.concat([df, pd.DataFrame([total_row])], ignore_index=True)
        # Aplicar formato dólar solo a columnas permitidas
# Función principal por hoja
def procesar_hoja(sheet):
    hoja_nombre = sheet.title
    headers = [cell.value for cell in sheet[1]]
    data = [
        [cell.value for cell in row]
        for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row)
        if any(cell.value is not None for cell in row)
    ]
    if not data:
        return

    df = pd.DataFrame(data, columns=headers)

    # Ordenar si existen
    ordenar_por = [col for col in ['FACULTAD', 'ENTRADA'] if col in df.columns]
    if ordenar_por:
        df = df.sort_values(by=ordenar_por)

    # Eliminar fila TOTAL si ya existía
    df = df[df[headers[0]] != 'TOTAL']

    # Agregar fila TOTAL con lógica especial
    df = agregar_fila_total_unicos(df, hoja_nombre)

    # Limpiar y reescribir
    sheet.delete_rows(1, sheet.max_row)

    # Insertar fila vacía al inicio
    sheet.insert_rows(1)

    # Insertar columna vacía al inicio
    sheet.insert_cols(1)

    # Insertar título
    sheet.insert_rows(2)
    sheet.cell(row=2, column=2).value = f"SEGUIMIENTO {hoja_nombre}"
    sheet.merge_cells(start_row=2, start_column=2, end_row=2, end_column=len(headers) + 1)
    sheet.cell(row=2, column=2).alignment = Alignment(horizontal='center', vertical='center')

    # Encabezados
    for col_idx, header in enumerate(headers, start=2):
        sheet.cell(row=3, column=col_idx, value=header)

    # Escribir filas
    for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=False), start=4):
        for c_idx, value in enumerate(row, start=2):  # empieza desde columna 2
            sheet.cell(row=r_idx, column=c_idx, value=value)

    # Combinar columnas clave
    for col in ['FACULTAD', 'META FACULTAD', 'ENTRADA']:
        combinar_y_centrar(sheet, col_name=col, header_row=3)
# Procesar todas las hojas
for hoja in wb.sheetnames:
    procesar_hoja(wb[hoja])
nombre_archivo="REPORTE_MATRICULADOS_AJUSTADO.xlsx"
wb.save(nombre_archivo)
#files.download(nombre_archivo)

In [ ]:
# Ruta y formato ya definidos
ruta_archivo = "REPORTE_MATRICULADOS_AJUSTADO.xlsx"
formato_moneda_excel ='_ "$"* #.##0,00_ ;_ "$"* -#.##0,00_ ;_ "$"* "-"??_ ;_ @_ '
# Abre el libro
wb = load_workbook(ruta_archivo)

# Columnas a formatear
columnas = ['RECAUDACIÓN', 'META PRESUPUESTO', 'PROMEDIO PAGO']

for ws in wb.worksheets:
    # Mapea encabezados (fila 2) a índices de columna
    headers = {}
    for cell in ws[3]:
        if cell.value in columnas:
            headers[cell.value] = cell.column  # número de columna

    # Aplica el formato moneda a todas las filas de datos
    for col_name, col_idx in headers.items():
        letra = get_column_letter(col_idx)
        for fila in range(3, ws.max_row + 1):
            ws[f"{letra}{fila}"].number_format = formato_moneda_excel

# Guarda el resultado
wb.save("REPORTE_MATRICULADOS_FORMATO_MONEDA.xlsx")

#descarga
#files.download("REPORTE_MATRICULADOS_FORMATO_MONEDA.xlsx")


####  </font> <font color='264CC7'> Aplicar formato (colores, ancho, negrita, dolar, porcentaje)  </font>


In [ ]:
# Cargar archivo base
ruta_archivo = "REPORTE_MATRICULADOS_FORMATO_MONEDA.xlsx"
wb = load_workbook(ruta_archivo)

# Estilos
fondo_azul = PatternFill(start_color="002060", end_color="002060", fill_type="solid")
letra_blanca_bold = Font(color="FFFFFF", bold=True)
letra_negrita = Font(bold=True)
centrado = Alignment(horizontal="center", vertical="center", wrap_text=False)
centrado_con_wrap = Alignment(horizontal="center", vertical="center", wrap_text=True)
borde_fino = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

# Formatos numéricos
formato_moneda_excel = '"$"#,##0.00'
formato_porcentaje = '0%'

# Columnas monetarias válidas
columnas_monetarias = ['RECAUDACIÓN', 'META PRESUPUESTO', 'PROMEDIO PAGO']

# Aplicar formato a todas las hojas
for hoja in wb.sheetnames:
    ws = wb[hoja]
    max_col = ws.max_column
    max_row = ws.max_row
    headers = [cell.value for cell in ws[2]]

    # 1. Fila título, encabezados y total
    for row in [2, 3, max_row]:
        for col in range(2, max_col + 1):
            cell = ws.cell(row=row, column=col)
            cell.fill = fondo_azul
            cell.font = letra_blanca_bold
            cell.alignment = centrado_con_wrap if ' ' in str(cell.value) else centrado
            cell.border = borde_fino

    # 2. Altura de fila de encabezado
    ws.row_dimensions[2].height = 34

    # 3. Ancho de columnas y alineaciones
    for col in range(2, max_col + 1):
        col_letter = get_column_letter(col)
        header = str(ws.cell(row=3, column=col).value)

        if ' ' in header:
            max_word_len = max(len(palabra) for palabra in header.split())
            ws.column_dimensions[col_letter].width = max_word_len + 5
            if header == 'PROMEDIO PAGO':
                ws.column_dimensions[col_letter].width += 2
            ws.cell(row=3, column=col).alignment = centrado_con_wrap

        if '%' in header:
            ws.column_dimensions[col_letter].width = (ws.column_dimensions[col_letter].width or 10) + 2

        if ' ' not in header and '%' not in header:
            max_len = len(header)
            for row in range(3, max_row + 1):
                val = ws.cell(row=row, column=col).value
                if val:
                    max_len = max(max_len, len(str(val)))
                ws.cell(row=row, column=col).alignment = centrado
                ws.cell(row=row, column=col).border = borde_fino
            ws.column_dimensions[col_letter].width = max_len + 4
            if header == 'RECAUDACIÓN':
                ws.column_dimensions[col_letter].width += 2

    # 4. Bordes en todo el contenido
    for row in range(3, max_row + 1):
        for col in range(2, max_col + 1):
            cell = ws.cell(row=row, column=col)
            cell.border = borde_fino

    # 5. Negrita en FACULTAD (excepto PUCETEC) y ENTRADA (en GRADO)
    if hoja.upper() != "PUCETEC" and 'FACULTAD' in headers:
        idx_facultad = headers.index("FACULTAD") + 1
        for row in range(3, max_row):
            ws.cell(row=row, column=idx_facultad).font = letra_negrita

    if hoja.upper().startswith("GRADO") and 'ENTRADA' in headers:
        idx_entrada = headers.index("ENTRADA") + 1
        for row in range(3, max_row):
            ws.cell(row=row, column=idx_entrada).font = letra_negrita

    # 6. Formato monetario (sin META, sin PROMEDIO PAGO en PUCETEC)
    for col_name in columnas_monetarias:
        if col_name in headers:
            if hoja.upper() == 'PUCETEC' and col_name == 'PROMEDIO PAGO':
                continue  # saltar PROMEDIO PAGO en PUCETEC
            col_idx = headers.index(col_name) + 1
            for row in range(4, max_row + 1):
                cell = ws.cell(row=row, column=col_idx)
                if isinstance(cell.value, (int, float)):
                    cell.number_format = formato_moneda_excel
                    cell.alignment = centrado

    # 7. Formato % sin decimales
    for col in range(1, max_col + 1):
        header = str(ws.cell(row=3, column=col).value)
        if '%' in header:
            for row in range(4, max_row + 1):
                cell = ws.cell(row=row, column=col)
                if isinstance(cell.value, str) and cell.value.endswith('%'):
                    try:
                        valor = float(cell.value.strip('%')) / 100
                        cell.value = valor
                    except ValueError:
                        continue
                if isinstance(cell.value, (int, float)):
                    cell.number_format = formato_porcentaje
                    cell.alignment = centrado

# Guardar archivo
archivo_nombre = "REPORTE_MATRICULADOS_FORMATEADA.xlsx"
wb.save(archivo_nombre)
#files.download("REPORTE_MATRICULADOS_FORMATEADA.xlsx")

####  </font> <font color='264CC7'> Formato Condicional % y Descarga </font>

In [ ]:
# Cargar el archivo
ruta_archivo = "REPORTE_MATRICULADOS_FORMATEADA.xlsx"
wb = load_workbook(ruta_archivo)

# Regla de color para escala verde → amarillo → rojo
regla_colores = ColorScaleRule(
    start_type='min', start_color='F8696B',
    mid_type='percentile', mid_value=50, mid_color='FFEB84',
    end_type='max', end_color='63BE7B'
)

# Encabezados objetivo
encabezados_porcentaje = [
    "% CUMPLIMIENTO POR CARRERA",
    "% CUMPLIMIENTO POR FACULTAD",
    "% CUMPLIMIENTO PRESUPUESTO",
]

# Aplicar formato condicional solo a celdas con contenido en %
for hoja in wb.sheetnames:
    ws = wb[hoja]
    headers = [cell.value for cell in ws[3]]  # Fila de encabezados está en la 3
    max_row = ws.max_row

    for col_idx, header in enumerate(headers, start=1):
        if isinstance(header, str) and header.strip().upper() in [h.upper() for h in encabezados_porcentaje]:
            col_letter = get_column_letter(col_idx)
            # Confirmar si hay valores numéricos antes de aplicar
            hay_numeros = any(
                isinstance(ws.cell(row=r, column=col_idx).value, (int, float))
                for r in range(4, max_row)
            )
            if hay_numeros:
                rango = f"{col_letter}4:{col_letter}{max_row - 1}"  # hasta antes de TOTAL
                ws.conditional_formatting.add(rango, regla_colores)

# Guardar resultado
archivo_final = "REPORTE_MATRICULADOS_FINAL.xlsx"
wb.save(archivo_final)

###</font> <font color='264CC7'> Descarga del Reporte Generado  </font>

In [ ]:
# Descargar Resultado
files.download(archivo_final)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>